In [1]:
import  glob
import  os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import  librosa
import  librosa.display
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import tensorflow as tf
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.layers.embeddings import Embedding
from keras.utils.np_utils import to_categorical
from keras.layers import Input, LSTM, GRU, Dense
from keras.layers import Bidirectional


Using TensorFlow backend.


In [3]:
ROOT_DIRECTORY = 'dataset/'

declarative_CLASS_train = 'NLP_Declarative/NLP_train_16k/*.wav'
question_CLASS_train = 'NLP_Question/NLP_train_16k/*.wav'

declarative_CLASS_test = 'NLP_Declarative/NLP_test_16k/*.wav'
question_CLASS_test = 'NLP_Question/NLP_test_16k/*.wav'


declarative_CLASS_val = 'NLP_Declarative/NLP_val_16k/*.wav'
question_CLASS_val = 'NLP_Question/NLP_val_16k/*.wav'


train_percentile = 0.8
test_percentile = 0.2

In [4]:
def filenames_list(directory_path):
    filename_list=[]
    for filename in glob.glob(directory_path):
        (filename_list.append(filename))
    return filename_list

In [ ]:
dec_files_list_train=filenames_list(ROOT_DIRECTORY+declarative_CLASS_train)
ques_files_list_train=filenames_list(ROOT_DIRECTORY+question_CLASS_train)


In [ ]:
dec_files_list_test=filenames_list(ROOT_DIRECTORY+declarative_CLASS_test)
ques_files_list_test=filenames_list(ROOT_DIRECTORY+question_CLASS_test)


In [ ]:
dec_files_list_val=filenames_list(ROOT_DIRECTORY+declarative_CLASS_val)
ques_files_list_val=filenames_list(ROOT_DIRECTORY+question_CLASS_val)

In [ ]:
# print(len(dec_files_list_train), len(dec_files_list_test), len(dec_files_list_val))
# print(len(ques_files_list_train), len(ques_files_list_test), len(ques_files_list_val))

In [ ]:
def extract_mfcc(file_name):
    X, sample_rate = librosa.load(file_name)
#     print ("Features :",len(X), X.shape, "sampled at ", sample_rate, "hz")
#     stft = np.abs(librosa.stft(X))
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
#     chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
#     mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
#     contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
#     tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs

In [5]:
Max_RNN=500


In [ ]:
def parse_audio_files(filename_list, label, Max_RNN):
    max_len=-999999
    Xs_train=[]
    Xs_test=[]
#     probability = [float(v) for v in percentiles.values()]
    
#     features, labels = np.empty((0,193)), np.empty(0)
    for e,fn in enumerate(filename_list):
        print(e, fn)
        mfccs = extract_mfcc(fn)
        if len(mfccs[0])> max_len:
            max_len=len(mfccs[0])
        mfccs=np.pad(mfccs, [(0, 0), (0, Max_RNN-len(mfccs[0]))], mode='constant')
        Xs_train.append(mfccs.T)
        
#         portion =  np.random.choice(2, 1, p=probability)
#         if portion == 0:
#             Xs_train.append(mfccs.T)
#         elif portion == 1:
#             Xs_test.append(mfccs.T)
            
#         print(mfccs.shape)
#         Xs_train.append(mfccs.T)
    #### Label for training
    if label == 'declarative':
        Ys_train = to_categorical(np.ones(len(Xs_train)),2)
#         Ys_test = to_categorical(np.ones(len(Xs_test)),2)
        
    elif label == 'question':
        Ys_train = to_categorical(np.zeros(len(Xs_train)),2)
#         Ys_test = to_categorical(np.zeros(len(Xs_test)),2)
    
    Xs_train = np.array(Xs_train)
#     Xs_test = np.array(Xs_test)
    return Xs_train, Ys_train, max_len 
        
#     for label, sub_dir in enumerate(sub_dirs):
#         for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
#             try:
#                 mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
#                 ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
#                 features = np.vstack([features,ext_features])
#                 labels = np.append(labels, fn.split('fold')[1].split('-')[1])
#             except:
#                 print("Error processing " + fn + " - skipping")
#     return np.array(features), np.array(labels, dtype = np.int)

### Creating mfcc vectors for declarative and questions

In [ ]:
# dec_Xs_train, dec_Ys_train, dec_max_len=parse_audio_files(dec_files_list_train, "declarative", Max_RNN)




In [ ]:
# dec_Xs_test, dec_Ys_test, dec_max_len_test=parse_audio_files(dec_files_list_test, "declarative", Max_RNN)
# dec_Xs_val, dec_Ys_val, dec_max_len_val=parse_audio_files(dec_files_list_val, "declarative", Max_RNN)

In [ ]:
# ques_Xs_train, ques_Ys_train, ques_max_len_train=parse_audio_files(ques_files_list_train, "question", Max_RNN)

In [ ]:
# ques_Xs_test, ques_Ys_test, ques_max_len_test=parse_audio_files(ques_files_list_test, "question", Max_RNN)
# ques_Xs_val, ques_Ys_val, ques_max_len_val=parse_audio_files(ques_files_list_val, "question", Max_RNN)

In [ ]:
# percentiles = {'train':train_percentile, 'test':test_percentile}
# dec_Xs_train, dec_Xs_test, dec_Ys_train, dec_Ys_test, dec_max_len=parse_audio_files(percentiles, dec_files_list, "declarative", Max_RNN)

# print(len(dec_Xs_train), len(dec_Ys_train), len(dec_Xs_test), len(dec_Ys_test), "Max Len: ", dec_max_len)

### Write data into files

In [ ]:
def write_file(file_name, vector_name):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for val in vector_name:
#             print(val)
#             sn, sr=librosa.load(fname_list[i], sr=None)
#             Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
# #             mag_Sn=np.abs(Sn)
#     #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, val, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            



In [ ]:
write_file("dec_Xs_val_full_mfcc.txt", dec_Xs_val)
write_file("ques_Xs_val_full_mfcc.txt", ques_Xs_val)
# write_file("train_n.txt", fname_trn)
# write_file("train_x.txt", fname_trx)

In [ ]:
write_file("dec_Xs_test_full_mfcc.txt", dec_Xs_test)
write_file("ques_Xs_test_full_mfcc.txt", ques_Xs_test)

In [ ]:
write_file("dec_Xs_train_full_mfcc.txt", dec_Xs_train)


In [ ]:
write_file("ques_Xs_train_full_mfcc.txt", ques_Xs_train)

### Read data from files

In [6]:
### Function for Reading file

def read_file(file_name):
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 500:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=float, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
        return sentence_full


In [9]:
dec_Xs_val_file = read_file("dec_Xs_val_full_mfcc.txt")
ques_Xs_val_file = read_file("ques_Xs_val_full_mfcc.txt")



332163
383265


In [ ]:
# print(len(dec_Xs_val_file), dec_Xs_val_file[0].shape)
# print(len(ques_Xs_val_file), ques_Xs_val_file[0].shape)

In [12]:
dec_Xs_test_file = read_file("dec_Xs_test_full_mfcc.txt")
ques_Xs_test_file = read_file("ques_Xs_test_full_mfcc.txt")


1430856
1379754


In [8]:
dec_Xs_train_file = read_file("dec_Xs_train_full_mfcc.txt")


6592158


In [10]:
ques_Xs_train_file = read_file("ques_Xs_train_full_mfcc.txt")


5876730


### Create Label data for 2 classes

In [13]:
dec_Ys_train_file = to_categorical(np.ones(len(dec_Xs_train_file)),2)
dec_Ys_test_file = to_categorical(np.ones(len(dec_Xs_test_file)),2)
dec_Ys_val_file = to_categorical(np.ones(len(dec_Xs_val_file)),2)

In [14]:
ques_Ys_train_file = to_categorical(np.zeros(len(ques_Xs_train_file)),2)
ques_Ys_test_file = to_categorical(np.zeros(len(ques_Xs_test_file)),2)
ques_Ys_val_file = to_categorical(np.zeros(len(ques_Xs_val_file)),2)

In [ ]:
print(dec_Xs_val_file[0].shape)
len(dec_Xs_val_file)

In [ ]:
# for i in dec_Xs_val_file:
#     print(i)

In [ ]:
# ques_Xs_train, ques_Xs_test, ques_Ys_train, ques_Ys_test, ques_max_len=parse_audio_files(percentiles, ques_files_list, "question", Max_RNN)

# print(len(ques_Xs_train), len(ques_Ys_train), len(ques_Xs_test), len(ques_Ys_test), "Max Len: ", ques_max_len)

In [15]:
# bx = np.concatenate( (dec_Xs_train,ques_Xs_train), axis = 0)
# by = np.concatenate( (dec_Ys_train,ques_Ys_train), axis = 0)

bx = np.concatenate( (dec_Xs_train_file,ques_Xs_train_file), axis = 0)
by = np.concatenate( (dec_Ys_train_file,ques_Ys_train_file), axis = 0)


In [16]:
# bx_val = np.concatenate( (dec_Xs_val,ques_Xs_val), axis = 0)
# by_val = np.concatenate( (dec_Ys_val,ques_Ys_val), axis = 0)

bx_val = np.concatenate( (dec_Xs_val_file,ques_Xs_val_file), axis = 0)
by_val = np.concatenate( (dec_Ys_val_file,ques_Ys_val_file), axis = 0)


In [17]:
n = bx.shape[0]
suffle_n = np.random.permutation(n)

bx = bx[suffle_n]
by = by[suffle_n]

In [18]:
n = bx_val.shape[0]
suffle_n = np.random.permutation(n)

bx_val = bx_val[suffle_n]
by_val = by_val[suffle_n]

In [16]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.33
set_session(tf.Session(config=config))

### Model

In [19]:
# Max_RNN = 1500

# create the model

model = Sequential()
model.add(Bidirectional(GRU(Max_RNN, return_sequences=False), input_shape=(Max_RNN,40)))
model.add(Dropout(0.2))

model.add(Dense(2, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(bx, by, validation_data = (bx_val, by_val), shuffle=True, nb_epoch=20, batch_size=64)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 1000)              1623000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2002      
Total params: 1,625,002
Trainable params: 1,625,002
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 24888 samples, validate on 1428 samples
Epoch 1/20
24888/24888 [==============================] - 422s 17ms/step - loss: 0.1198 - acc: 0.9462 - val_loss: 0.0536 - val_acc: 0.9832
Epoch 2/20
24888/24888 [==============================] - 401s 16ms/step - loss: 0.0340 - acc: 0.9879 - val_loss: 0.0570 - val_acc: 0.9776
Epoch 3/20
24888/24888 [==============================] - 401s 16ms/step - loss: 0.0276 - acc: 0.9905 - val_loss: 0.0437 - val_acc: 0.9860
Epoch 4/20
24888/24888 [==============================] - 402s 16ms/step - loss: 0.0242 - acc: 0.9916 - val_loss: 0.0524 - val_acc: 0.9832
Epoch 5/20
24888/24888 [==============================] - 395s 16ms/step - loss: 0.0327 - acc: 0.9886 - val_loss: 0.0471 - val_acc: 0.9860
Epoch 6/20
24888/24888 [==============================] - 400s 16ms/step - loss: 0.0247 - acc: 0.9908 - val_loss: 0.0555 - val_acc: 0.9811
Epoch 7/20
24888/24888 [==============================] - 407s 16ms/step - loss: 0.0211 - acc: 0.9926 - val_loss: 0.0

In [21]:
bx_test = np.concatenate( (dec_Xs_test_file, ques_Xs_test_file), axis = 0)
by_test = np.concatenate( (dec_Ys_test_file, ques_Ys_test_file), axis = 0)


scores = model.evaluate(bx_test, by_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 96.95%


In [ ]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        X,sr = librosa.load(fp)
        print(X.shape)
        raw_sounds.append(X)
    return raw_sounds

def plot_waves(raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,10), dpi = 900)
    for f in raw_sounds:
        plt.subplot(2,5,i)
        librosa.display.waveplot(np.array(f),sr=22050)
        i += 1
    plt.suptitle('Figure 1: Waveplot',x=0.5, y=0.95,fontsize=18)
    plt.show()
    
def plot_specgram(raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,10), dpi = 900)
    for f in raw_sounds:
        plt.subplot(2,5,i)
        specgram(np.array(f), Fs=22050)
        i += 1
    plt.suptitle('Figure 2: Spectrogram',x=0.5, y=0.95,fontsize=18)
    plt.show()


In [ ]:

raw_sounds_ques = load_sound_files(ques_files_list[:5])
plot_waves(raw_sounds_ques)
# plot_specgram(raw_sounds_ques)
# raw_sounds_dec = load_sound_files(dec_files_list[:5])
# plot_waves(raw_sounds_dec)
# plot_specgram(raw_sounds_dec)

In [ ]:
plot_specgram(raw_sounds_ques)


In [ ]:
raw_sounds_dec = load_sound_files(dec_files_list[:5])
plot_waves(raw_sounds_dec)

In [ ]:
plot_specgram(raw_sounds_dec)

In [ ]:
sample_filename = "Corpus/A_man_bought_a_car_in_Paris-DC-2.wav"
mfccs=extract_mfcc(sample_filename)
print ("MFCSS  = ", len(mfccs))
data_points, _ = librosa.load(sample_filename)
print( "IN: Initial Data Points =", len(data_points), np.shape(data_points))
print ("OUT: Total features =",  mfccs.shape)

In [ ]:
mfccs=np.pad(mfccs, [(0, 0), (0, RNN_LENGTH-len(mfccs[0]))], mode='constant')
print(mfccs.shape)
mfccs[0]

In [ ]:
resulted_list=parse_audio_files(filename_list)
len(resulted_list)


In [ ]:
len(raw_sounds)
for i in range(len(raw_sounds)):
    print(raw_sounds[i].shape)

In [ ]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    print ("Features :",len(X), X.shape, "sampled at ", sample_rate, "hz")
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
    return mfccs,chroma,mel,contrast,tonnetz

def parse_audio_files(parent_dir,sub_dirs,file_ext='*.wav'):
    features, labels = np.empty((0,193)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            try:
                mfccs, chroma, mel, contrast, tonnetz = extract_feature(fn)
                ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
                features = np.vstack([features,ext_features])
                labels = np.append(labels, fn.split('fold')[1].split('-')[1])
            except:
                print("Error processing " + fn + " - skipping")
    return np.array(features), np.array(labels, dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

def assure_path_exists(path):
    mydir = os.path.join(os.getcwd(), path)
    if not os.path.exists(mydir):
        os.makedirs(mydir)